In [62]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

PATH="Assets/Data/"

In [63]:
import json
import pandas as pd

with open(PATH+"swarmData.json", "r") as f:
    data = json.load(f)

rows = []

# Read top-level constants (if you want them in every row)
max_speed = data.get("maxSpeed", None)
max_force = data.get("maxForce", None)
neighbor_radius = data.get("neighborRadius", None)
alpha = data.get("alpha", None)
beta = data.get("beta", None)
delta = data.get("delta", None)
avoidance_radius = data.get("avoidanceRadius", None)
avoidance_force = data.get("avoidanceForce", None)
drone_radius = data.get("droneRadius", None)
damping_factor = data.get("dampingFactor", None)

In [66]:

rows = []

for drone_entry in data["swarmState"]:
    drone_id = int(drone_entry["droneId"])
    drone_state = drone_entry["droneState"]

    positions = drone_state.get("position", [])
    velocities = drone_state.get("velocity", [])
    obstacles = drone_state.get("obstacleAvoidance", [])
    olfati = drone_state.get("olfatiSaber", [])
    embodied_list = drone_state.get("embodied", [])
    selected_list = drone_state.get("selected", [])
    network_list = drone_state.get("network", [])

    # Determine how many time steps exist for this drone
    number_of_time_steps = max(
        len(positions),
        len(velocities),
        len(embodied_list),
        len(selected_list),
        len(network_list)
    )

    for i in range(number_of_time_steps):
        # Fetch the position dictionary if it exists, otherwise {}
        if i < len(positions) and isinstance(positions[i], dict):
            pos = positions[i]
            posX = pos.get("x", None)
            posY = pos.get("y", None)
            posZ = pos.get("z", None)
        else:
            posX, posY, posZ = None, None, None

        # Fetch the velocity dictionary if it exists, otherwise {}
        if i < len(velocities) and isinstance(velocities[i], dict):
            vel = velocities[i]
            velX = vel.get("x", None)
            velY = vel.get("y", None)
            velZ = vel.get("z", None)
        else:
            velX, velY, velZ = None, None, None

        if i < len(obstacles) and isinstance(obstacles[i], dict):
            obs = obstacles[i]
            obsX = obs.get("x", None)
            obsY = obs.get("y", None)
            obsZ = obs.get("z", None)

        if i < len(olfati) and isinstance(olfati[i], dict):
            olf = olfati[i]
            olfX = olf.get("x", None)
            olfY = olf.get("y", None)
            olfZ = olf.get("z", None)

        # Booleans or other single-value lists
        emb = embodied_list[i] if i < len(embodied_list) else None
        sel = selected_list[i] if i < len(selected_list) else None
        net = network_list[i] if i < len(network_list) else None

        row_data = {
            "droneId": drone_id,
            "timeStep": i,
            "posX": posX,
            "posY": posY,
            "posZ": posZ,
            "velX": velX,
            "velY": velY,
            "velZ": velZ,
            "obsX": obsX,
            "obsY": obsY,
            "obsZ": obsZ,
            "olfX": olfX,
            "olfY": olfY,
            "olfZ": olfZ,
            "embodied": emb,
            "selected": sel,
            "network": net
        }
        rows.append(row_data)

df = pd.DataFrame(rows)

In [75]:
droneDied = []

df.groupby("droneId").count()[["timeStep"]]

,timeStep
droneId,
0,1463
1,1463
2,468
3,473
4,1463
5,473
6,1463
7,1463
8,1463


In [53]:
df['network'].unique()

array([''], dtype=object)